In [2]:
import numpy as np  
from matplotlib import pyplot as plt 
from PIL import Image    
import torch

from anomalib.config import get_configurable_parameters
from anomalib.data import get_datamodule
from anomalib.models import get_model
from anomalib.models.components import feature_extractors
import torchvision
from anomalib.models.components.feature_extractors import TorchFXFeatureExtractor
from torchvision.models.densenet import DenseNet201_Weights
import torch.nn.functional as F
from anomalib.models.components.cluster.kmeans import KMeans
import torchvision.models as models

/work/scratch/tyang/miniconda3/env/anomalib_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL = "patchcore"
CONFIG_PATH = f"/home/students/tyang/anomalib/src/anomalib/models/{MODEL}/config.yaml"
with open(file=CONFIG_PATH, mode="r",encoding="utf-8") as f:
    print(f.read())
    
config = get_configurable_parameters(config_path=CONFIG_PATH)


dataset:
  name: airogs
  format: airogs
  path: /home/students/tyang/airogs 
  task: classification # options: [classification, segmentation]
  category: 0
  number_of_samples: 17999
  pre_selection: False
  train_batch_size: 1000
  eval_batch_size: 32
  num_workers: 8
  image_size: 256 # dimensions to which images are resized (mandatory)
  center_crop: 224 # dimensions to which images are center-cropped after resizing (optional)
  normalization: imagenet # data distribution to which the images will be normalized: [none, imagenet]
  transform_config:
    train: null
    eval: null
  test_split_mode: from_dir # options: [from_dir, synthetic]
  test_split_ratio: 0.2 # fraction of train images held out testing (usage depends on test_split_mode)
  val_split_mode: same_as_test # options: [same_as_test, from_test, synthetic]
  val_split_ratio: 0.2 # fraction of train/test images held out for validation (usage depends on val_split_mode)
  tiling:
    apply: false
    tile_size: null
    stri

/home/students/tyang/anomalib/src/anomalib/config/config.py:243: UserWarning: The seed value is now fixed to 0. Up to v0.3.7, the seed was not fixed when the seed value was set to 0. If you want to use the random seed, please select `None` for the seed value (`null` in the YAML file) or remove the `seed` key from the YAML file.
  warn(
/home/students/tyang/anomalib/src/anomalib/config/config.py:280: UserWarning: config.project.unique_dir is set to False. This does not ensure that your results will be written in an empty directory and you may overwrite files.
  warn(


In [4]:

data_module = get_datamodule(config=config)
data_module.prepare_data() # check if the dataset is avaliable
data_module.setup()

i, train_data = next(enumerate(data_module.train_dataloader()))


In [5]:

feature_extractor = TorchFXFeatureExtractor(
                    backbone="densenet201",
                    return_nodes=["features.denseblock1.denselayer6.conv2"],
                    weights=DenseNet201_Weights.IMAGENET1K_V1,
                )


/work/scratch/tyang/miniconda3/env/anomalib_env/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/work/scratch/tyang/miniconda3/env/anomalib_env/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/work/scratch/tyang/miniconda3/env/anomalib_env/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/work/scratch/tyang/miniconda3/env/anomalib_env/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


In [6]:
# feature shape is (batch_size, channel, height, width) 
feature = feature_extractor(train_data["image"])
print(feature["features.denseblock1.denselayer6.conv2"].shape)
print(train_data["image"].shape)

torch.Size([1000, 32, 56, 56])
torch.Size([1000, 3, 224, 224])


In [7]:
# extract features from the whol training dataset

feature_list = []

for  i, train_data in enumerate(data_module.train_dataloader()):
    features = feature_extractor(train_data["image"])["features.denseblock1.denselayer6.conv2"]
    feature_list.append(features)



In [10]:
# convert the list of features to a tensor
global_feature_tensor = torch.vstack(feature_list)   

In [11]:
# global feature shape is (train_data size, channel, height, width)
print(global_feature_tensor.shape)

torch.Size([13918, 32, 56, 56])


In [12]:
from anomalib.models.components.cluster.kmeans import KMeans

def get_kmeans_centers(feature_t, n_clusters):
    """
    Args:
        feature_t: feature tensor, shape is (batch_size, channel, height, width)
        n_clusters: number of clusters
        
    Returns:
        cluster_center: shape is (n_clusters, channel)
        kmeans: kmeans model   """
    
    feature_t = feature_t.permute(1,0,2,3)
    feature_t = feature_t.flatten(start_dim=1)
    feature_t= feature_t.permute(1,0)


    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(feature_t)
    cluster_center = kmeans.cluster_centers_
   
    return cluster_center, kmeans



In [13]:
clusters_centers,kmeans = get_kmeans_centers(global_feature_tensor, n_clusters=12)

print(clusters_centers.shape)
print(len(clusters_centers))

torch.Size([12, 32])
12


In [15]:
def bag_of_words_statistics(Ptst, Cref, S):
    """
    Params:
    Ptst: Feature tensor of a set of images, tensor of shape (N, C, H, W)
    Cref: reference Cluster centers, tensor of shape (K, C)
    S: number of subregions per image dimension, integer

    Returns:
    bow_stats: list of normalized Bag-of-words statistics, possibility-like , length N, each element is a tensor of shape (S * S, K)
    """
    Ptst = torch.vsplit(Ptst, Ptst.shape[0])
    bow_stats = []
    for Itst in Ptst:
        Itst = Itst.squeeze(0)
        
        subtensors = torch.chunk(Itst, S, dim=1)
        subtensor = [torch.chunk(st, S, dim=2) for st in subtensors]
        
        
        image_bow_stats = torch.zeros(S * S, len(Cref), dtype=torch.float32)
        for i in range(S):
            for j in range(S):
                new_subtensor = subtensor[i][j]
                
                new_subtensor = new_subtensor.flatten(start_dim=1)
                
                new_subtensor = new_subtensor.permute(1,0)
                
                cluster_idx = kmeans.predict(new_subtensor)
                
                cluster_idx = cluster_idx.float()
                hist = torch.histc(cluster_idx, bins = len(Cref), min = torch.min(cluster_idx), max = torch.max(cluster_idx))
                normalized_hist = hist / torch.sum(hist)
                
                image_bow_stats[i * S + j] = normalized_hist
        

        bow_stats.append(image_bow_stats)
            
        
    return bow_stats


In [16]:
global_bow_stats = bag_of_words_statistics(global_feature_tensor, clusters_centers, S=4)

In [17]:
print(len(global_bow_stats))
print(global_bow_stats[0].shape)

13918
torch.Size([16, 12])


In [18]:
import torch.nn.functional as F

def kl_distance(i_hist, j_hist):
    """Params:
    i_hist: bow histogram of image i, tensor of shape (1, s * s, K )
    j_hist: bow histogram of image j, tensor of shape (1, s * s, K )

    Returns:
    kl_dist: kl distance between image i and image j, tensor of shape (1)
    """
    i_hist[i_hist == 0] = 1e-10
    j_hist[j_hist == 0] = 1e-10
    kl_divegence = F.kl_div(j_hist.log(), i_hist,reduction="none")
    
    sum = torch.sum(kl_divegence,dim=-1)
    topk,ids = torch.topk(sum, k=11, dim=0, largest=False, sorted=True)
   
    dist = torch.mean(topk)
  
    return dist
    

In [19]:
def kl_select(bow_stats, select_ratio, step_size):
    """
    Params:
    bow_stats: list of normalized Bag-of-words statistics, possibility-like , length N, each element is a tensor of shape (S * S, K)
    select_ratio: ratio of selected images compare to all images, float
    step_size: batch size, used for big taining dataset, integer

    Returns:
    selected_idx: list of selected image indices in each batch, length number of batches, each element has the size of batch_size * select_ratio
    """
    
    stacked_bow_stats = torch.vstack(bow_stats)
    stacked_bow_stats_reshaped = stacked_bow_stats.view(len(bow_stats),1,bow_stats[0].shape[0],bow_stats[0].shape[1])
    stacked_bow_stats_transposed = stacked_bow_stats_reshaped.permute(1,0,2,3)

    stacked_bow_stats_reshaped[stacked_bow_stats_reshaped == 0 ] = 1e-10
    stacked_bow_stats_transposed[stacked_bow_stats_transposed == 0 ] = 1e-10

    selected_idxs =[]
    selected_distances = []
    for i in range(0, stacked_bow_stats_reshaped.shape[0], step_size):
        current_bow = stacked_bow_stats_reshaped[i:i+step_size if i+step_size < stacked_bow_stats_reshaped.shape[0] else stacked_bow_stats_reshaped.shape[0], :, :, :]
        kl_divegence = F.kl_div(current_bow.log(), stacked_bow_stats_transposed,reduction="none")

        sum = torch.sum(kl_divegence,dim=3)
        topk,ids = torch.topk(sum, k=11, dim=2, largest=False, sorted=True)
        dist_matrix = torch.mean(topk,dim=2)
  
        dist_l = torch.sum(dist_matrix,dim=1)
        topk_far_distances,idx = torch.topk(dist_l, k=int(len(dist_l) * select_ratio), dim=0, largest=True, sorted=True)
     
        selected_idxs.append(idx)
        selected_distances.append(topk_far_distances)
    
    return selected_idxs, selected_distances

In [20]:
idxs,distances = kl_select(global_bow_stats, select_ratio=0.1, step_size=1000)

: 

In [ ]:
stack_distances = torch.cat(distances)
print(stack_distances.shape)

In [ ]:
def second_selection(index, distance, select_ratio):
    """
    Params:
    idxs: list of selected image indices in each batch , length number of batches
    distances: list of selected image distances, length number of batches
    select_ratio: ratio of selected images compare to all images, float

    Returns:
    selected_idx: list of selected image indices with biggest distances over all in each batches, length number of batches
    """
    selected_distances = [[] for _ in range(len(distance))]
    distance = torch.cat(distance)
    topk, global_ids = torch.topk(distance, k=int(len(distance) * select_ratio), dim=0, largest=True, sorted=True)
    
    selected_idxs = [[] for _ in range(len(index))]
    
    for global_id in global_ids:
        batch_id = global_id // index[0].shape[0]
        local_id = global_id % index[0].shape[0]
        selected_idxs[batch_id].append(index[batch_id][local_id])
        selected_distances[batch_id].append(distance[global_id])
        
    
    return selected_idxs, selected_distances

In [17]:
second_indices,second_dists = second_selection(idxs, distances, select_ratio=0.75)

NameError: name 'second_selection' is not defined

In [18]:
print(len(second_indices))
print(len(second_dists))
print(second_indices[0])
print(second_indices[0].shape)

14


In [19]:
merged_datas = {"image_path": [], "label": []}
for  i, train_data in enumerate(data_module.train_dataloader()):
    selected_data = ( {"image_path": train_data["image_path"][index], "label": train_data["label"][index]} for index in idxs[i] )
    
    
    for data in selected_data:
        for key, values in data.items():
            merged_datas[key].append(values)
    

print(merged_datas["image_path"])

['/home/students/tyang/airogs/0/TRAIN001072.jpg', '/home/students/tyang/airogs/0/TRAIN000970.jpg', '/home/students/tyang/airogs/0/TRAIN000580.jpg', '/home/students/tyang/airogs/0/TRAIN001220.jpg', '/home/students/tyang/airogs/0/TRAIN000946.jpg', '/home/students/tyang/airogs/0/TRAIN000845.jpg', '/home/students/tyang/airogs/0/TRAIN001168.jpg', '/home/students/tyang/airogs/0/TRAIN001163.jpg', '/home/students/tyang/airogs/0/TRAIN000967.jpg', '/home/students/tyang/airogs/0/TRAIN000975.jpg', '/home/students/tyang/airogs/0/TRAIN001088.jpg', '/home/students/tyang/airogs/0/TRAIN000585.jpg', '/home/students/tyang/airogs/0/TRAIN000158.jpg', '/home/students/tyang/airogs/0/TRAIN000718.jpg', '/home/students/tyang/airogs/0/TRAIN000822.jpg', '/home/students/tyang/airogs/0/TRAIN001177.jpg', '/home/students/tyang/airogs/0/TRAIN000870.jpg', '/home/students/tyang/airogs/0/TRAIN000015.jpg', '/home/students/tyang/airogs/0/TRAIN000841.jpg', '/home/students/tyang/airogs/0/TRAIN000093.jpg', '/home/students/tya

In [20]:
print(len(merged_datas["image_path"]))

1391


In [21]:
import csv 

csv_path = "/home/students/tyang/Documents/cpr_trainingdata.csv"


with open(csv_path, mode="w", newline="") as csv_file:
        fieldnames = ["image_path", "label"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        for i in range(len(merged_datas["image_path"])):
            rowdict = {"image_path": merged_datas["image_path"][i], "label": merged_datas["label"][i]}
            writer.writerow(rowdict)